# Multivariate

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os

Then we download the data base **dat**.

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(fileNF)
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Elevations.NF")
target = gl.DbGrid.createFromNF(fileNF)
ndim = 2
gl.ASpaceObject.defineDefaultSpace(gl.ESpaceType.SPACE_RN, ndim);

In [ ]:
dat.setName("January_temp","Temperature")
unique_neigh = gl.NeighUnique.create(ndim = ndim)

Statistics

In [ ]:
gl.dbStatisticsPrint(dat, ["Elevation", "Temperature"])
gl.dbStatisticsPrint(target, ["Elevation"])

Observations

In [ ]:
ax = gp.point(dat, color_name="Elevation", figsize=[8,8], aspect=1)

Map of the elevations

In [ ]:
ax = gp.grid(target, "Elevation", figsize=[8,8], aspect=1)

Temperature vs. Elevation

In [ ]:
ax = gp.correlation(dat, namex="Elevation", namey="Temperature", asPoint=True, regrLine=True,
                   title="Correlation between Temperature and Elevation")

Variograms and Non-stationarity

Directional variograms (N-S and E-W): no drift or with first order global trend

In [ ]:
varioparam = gl.VarioParam.createMultiple(ndim=ndim, ndir=2, npas=30, dpas=10)
vario_raw2dir = gl.Vario(varioparam, dat)
err = vario_raw2dir.compute()

model = gl.Model.create()
model.setDriftIRF(1)
vario_res2dir = gl.Vario(varioparam, dat)
err = vario_res2dir.compute(model=model)

axs = gp.vario(vario_raw2dir, title="Temperature (°C)", xlabel = "Distance (km)", ylabel = "Variogram")
axs = gp.vario(vario_res2dir,axs=axs, linestyle='dashed')

Global trend

Find the coefficients of the global regression (first order polynomial)

In [ ]:
err = gl.regression(dat, name0="Temperature", mode=2, model=model, verbose=True)

Modelling Variogram of Raw Data with Anisotropy

In [ ]:
fitmod = gl.Model()
err = fitmod.fit(vario_raw2dir,types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC])

In [ ]:
ax = gp.varmod(vario_raw2dir, fitmod, title="Temperature (°C)", 
               xlabel = "Distance (km)", ylabel = "Variogram", lw=2)

Ordinary Kriging - Cross-Validation

Perform Cross-validation (using Ordinary Kriging) and calculate the Mean Squared Error.

In [ ]:
err = gl.xvalid(dat, fitmod, unique_neigh, namconv=gl.NamingConvention("OK",True,True,False))
gl.dbStatisticsPrint(dat, ["OK.Temperature.*"])

Ordinary Kriging - Estimation

In [ ]:
err = gl.kriging(dat, target, fitmod, unique_neigh, namconv=gl.NamingConvention("OK"))

Graphic representation of Ordinary Kriging estimation

In [ ]:
ax = gp.grid(target, name="OK*estim", title="Temperature - Ordinary Kriging", figsize=[8,8], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Statistics on the Ordinary Kriging

In [ ]:
gl.dbStatisticsPrint(target, ["OK.Temperature.*"])

Fitting Variogram of Residuals

In [ ]:
fitmodUK = gl.Model()
err = fitmodUK.fit(vario_res2dir,types=[gl.ECov.SPHERICAL],optvar=gl.Option_VarioFit(False,False))

In [ ]:
ax = gp.varmod(vario_res2dir, fitmodUK, title="Temperature (°C)", 
               xlabel = "Distance (km)", ylabel = "Variogram", lw=2)

Note that the residuals seem isotropic, hence use isotropic option for Fitting.

Universal Kriging - Cross-Validation

Perform Cross-validation (using Universal Kriging) and calculate the Mean Squared Error.

In [ ]:
err = gl.xvalid(dat, fitmodUK, unique_neigh, namconv=gl.NamingConvention("UK",True,True,False))
gl.dbStatisticsPrint(dat, ["UK.Temperature.*"])

Universal Kriging - Estimation

In [ ]:
err = gl.kriging(dat, target, fitmodUK, unique_neigh, namconv=gl.NamingConvention("UK"))

Graphic representation

In [ ]:
ax = gp.grid(target, name="UK*estim", title="Temperature - Universal Kriging", figsize=[8,8], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Statistics on Universal Kriging

In [ ]:
gl.dbStatisticsPrint(target, ["UK.Temperature.*"])

Comparing Ordinary and Universal Krigings

In [ ]:
ax = gp.correlation(target, "OK*estim", "UK*estim", bissLine=True,
                    xlabel="Ordinary Kriging",ylabel="Universal Kriging")

Comparing Temperature and Elevation

In [ ]:
ax = gp.correlation(dat,"Elevation","Temperature",regrLine=True,asPoint=True)

Average of Elevations of all data and average of Elevations of meteorological stations

In [ ]:
gl.dbStatisticsPrint(dat, ["Elevation", "Temperature"])

Average of Elevations when Temperatures are defined

In [ ]:
sel = dat["Temperature"]
uid = dat.addSelection(~np.isnan(sel),"Temperature defined")
gl.dbStatisticsPrint(dat, ["Elevation", "Temperature"])

Bivariate Modelling

In [ ]:
dat.deleteColumn("Temperature defined")
dat.setLocators(["Temperature", "Elevation"], gl.ELoc.Z)

Bivariate Modelling

In [ ]:
varioexp2var = gl.Vario.create(varioparam, dat)
err = varioexp2var.compute()
fitmod2var = gl.Model()
err = fitmod2var.fit(varioexp2var,types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC])

In [ ]:
ax = gp.varmod(varioexp2var, fitmod2var, title="Temperature (°C) and Elevation", figsize=[5,5],
               xlabel = "Distance (km)", ylabel = "Variogram", lw=2)

Cokriging with elevation - Cross-Validation

Most of the processes are more time-consuming in Unique Neighborhood.
We create a small neighborhood for demonstration.

In [ ]:
moving_neigh = gl.NeighMoving.create(ndim=2, radius = 1000, nmaxi = 10)

Perform Cross-validation (Bivariate Model) and calculate the Mean Squared Error.

In [ ]:
err = gl.xvalid(dat, fitmod2var, moving_neigh, namconv=gl.NamingConvention("COK",True,True,False))
gl.dbStatisticsPrint(dat, ["COK.Temperature.*"])

Cokriging with elevation - Estimate

In [ ]:
err = gl.kriging(dat, target, fitmod2var, unique_neigh, namconv=gl.NamingConvention("COK"))

Graphic representation

In [ ]:
ax = gp.grid(target, name="COK.T*estim", title="Temperature - CoKriging (with Elevation)", figsize=[8,8], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Statistics

In [ ]:
gl.dbStatisticsPrint(target, ["COK.Temperature.*"])

Comparing Kriging and CoKriging

In [ ]:
ax = gp.correlation(target, "OK.T*estim", "COK.T*estim", bissLine=True,
                    xlabel="Ordinary Kriging",ylabel="Ordinary CoKriging")

Note that CoKriging produces estimates which are mostly larger than Kriging estimates.

Kriging the residuals

$$
Z_2(s)=b + a Z_1(s) + R(s)
$$

A first call to the regression function is done in order to retreive the coefficients of the regression

In [ ]:
regr = gl.regression(dat, "Temperature", ["Elevation"], 0, flagCste=True, verbose=True)
b = regr.coeffs[0]
a = regr.coeffs[1]

We calculate the Residuals and provide some statistics

In [ ]:
err = gl.dbRegression(dat, "Temperature", ["Elevation"],
                     namconv = gl.NamingConvention("Regr",True,True,False))
gl.dbStatisticsPrint(dat, ["Regr*"])

In [ ]:
ax = gp.correlation(dat,"Elevation","Regr*",regrLine=True,asPoint=True)

Kriging the residuals - Variogram of the residual

In [ ]:
dat.setLocator("Regr*",gl.ELoc.Z, cleanSameLocator=True)
varioexpR = gl.Vario(varioparam, dat)
err = varioexpR.compute()
fitmodR = gl.Model()
err = fitmodR.fit(varioexpR,types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.ORDER1_GC])

In [ ]:
ax = gp.varmod(varioexpR, fitmodR, title="Temperature Residual", 
               xlabel = "Distance (km)", ylabel = "Variogram", lw=2)

Kriging the residuals

In [ ]:
err = gl.kriging(dat, target, fitmodR, unique_neigh, namconv=gl.NamingConvention("ROK"))

In [ ]:
ax = gp.grid(target, name="ROK.Regr*estim", title="Temperature Residuals", figsize=[10,10], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Kriging the residuals - Computing the estimate

$$
Z_2^{\star} = b + a Z_1(s) + R(s)^{OK}
$$


In [ ]:
ROK_estim = target["ROK.Regr*estim"] + b + a * target["Elevation"]
uid = target.addColumns(ROK_estim,"KR.Temperature.estim")

In [ ]:
ax = gp.grid(target, name="KR.T*estim", title="Temperature with Residuals", figsize=[10,10], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Correlation between Ordinary Kriging and CoKriging

In [ ]:
ax = gp.correlation(target, "OK.T*estim", "COK.T*estim", bissLine=True,
                    xlabel="Ordinary Kriging",ylabel="Ordinary CoKriging")

Correlation between Ordinary Kriging and Kriging withResiduals

In [ ]:
ax = gp.correlation(target, "OK.T*estim", "KR.T*estim", bissLine=True,
                    xlabel="Ordinary Kriging",ylabel="Kriging with Residuals")

Some statistics for comparison

In [ ]:
gl.dbStatisticsPrint(target, ["OK.T*estim", "UK.T*estim", "COK.T*estim", "KR.T*estim"])

## Using Elevation Map as External Drift

Preparing the data bases

In [ ]:
dat.setLocator("Temperature",gl.ELoc.Z,cleanSameLocator=True)
dat.setLocator("Elevation",gl.ELoc.F,cleanSameLocator=True)

In [ ]:
varioKED = gl.Vario(varioparam, dat)

model = gl.Model.create()
model.setDriftIRF(order=0, nfex=1)
err = varioKED.compute(model=model)

Comparing the Experimental variograms

In [ ]:
axs = gp.vario(vario_raw2dir)
axs = gp.vario(varioKED,axs=axs, linestyle='dashed')

Model of Residuals (External Drift)

In [ ]:
modelKED = gl.Model()
err = modelKED.fit(varioKED,types=[gl.ECov.NUGGET, gl.ECov.CUBIC, gl.ECov.GAUSSIAN])
modelKED.setDriftIRF(order = 0, nfex = 1)

Cross-Validation with External Drift

In [ ]:
err = gl.xvalid(dat, modelKED, unique_neigh, namconv=gl.NamingConvention("KED",True,True,False))
gl.dbStatisticsPrint(dat, ["KED.Temperature.*"])

Kriging with External Drift

In [ ]:
err = gl.kriging(dat, target, modelKED, unique_neigh, namconv=gl.NamingConvention("KED"))

Graphic representation

In [ ]:
ax = gp.grid(target, name="KED.T*estim", title="Temperature with External Drift", figsize=[10,10], aspect=1)
ax = gp.point(dat,size_name="Temperature", ax=ax, flagSizeLegend=False)

Statistics on Kriging with Elevation as External Drift

In [ ]:
gl.dbStatisticsPrint(target, ["KED.T*"])

Comparing Ordinary Kriging and Kriging with External Drift

In [ ]:
ax = gp.correlation(target, "OK.T*estim", "KED.T*estim", bissLine=True,
                    xlabel="Ordinary Kriging",ylabel="Kriging with External Drift")

Note that negative Estimates are present when using External Drift.

## Summary of Cross-validation scores

Statistics on the cross-validation Mean Squared Errors (for Temperature)

In [ ]:
gl.dbStatisticsPrint(dat, ["*T*.esterr"])

Statistics on the various estimates

In [ ]:
gl.dbStatisticsPrint(target, ["*.Temperature.estim"])

Statistics on the various standard deviation of estimation errors

In [ ]:
gl.dbStatisticsPrint(target, ["*.Temperature.stdev"])